# Homework 1


### Imports

In [ ]:
import sklearn as sk
from sklearn.feature_selection import f_classif
from sklearn.model_selection import train_test_split
from sklearn import tree
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from scipy.io.arff import loadarff

: 

### Loading the data set 

In [ ]:
IMAGES_DIR = Path('images')
DATA_DIR = Path('data')
DATA_FILE = 'column_diagnosis.arff'
DATA_PATH = DATA_DIR / DATA_FILE
data = loadarff(DATA_PATH)
df = pd.DataFrame(data[0])
df['class'] = df['class'].str.decode('utf-8')
# Show the first 5 rows
df.head()

: 

# Question 1

In [ ]:
X = df.drop('class', axis=1)
y = df['class']
f_statistic, p_values = f_classif(X, y)

f_statistic_df = pd.DataFrame({'feature': X.columns,
                               'F': f_statistic,
                               'p-value': p_values})
f_statistic_df.sort_values('p-value')

highest_discriminative_feature = f_statistic_df['feature'][f_statistic_df['F'].idxmax()]
lowest_discriminative_feature = f_statistic_df['feature'][f_statistic_df['F'].idxmin()]

print(f_statistic_df.sort_values('p-value'))
print("\n")
print(f"Variable with the highest F-statistic and lowest p_value: {highest_discriminative_feature}\n")
print(f"Variable with the lowest F-statistic and highest p_value: {lowest_discriminative_feature}\n")


: 

In [ ]:
# Plot the class-conditional probability density functions for the two selected features
plt.figure(figsize=(12, 6))

for class_name in df['class'].unique():
    sns.kdeplot(df[df['class'] == class_name][highest_discriminative_feature], label=class_name)

plt.title(f"Class Conditional Probability Density Function for {highest_discriminative_feature}")
plt.xlabel(highest_discriminative_feature)
plt.ylabel('Probability density')
plt.legend()
plt.savefig(IMAGES_DIR / 'prob_dens_highest_discriminative_feature.png')
plt.show()

plt.figure(figsize=(12, 6))
for class_name in df['class'].unique():
    sns.kdeplot(df[df['class'] == class_name][lowest_discriminative_feature], label=class_name)

plt.title(f"Class Conditional Probability Density Function for {lowest_discriminative_feature}")
plt.xlabel(lowest_discriminative_feature)
plt.ylabel('Probability density')
plt.legend()
plt.savefig(IMAGES_DIR /'prob_dens_lowest_discriminative_feature.png')
plt.show()





: 

# Question 2


In [ ]:
# Load and partition data
X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    train_size=0.7,
                                                    random_state=0,
                                                    stratify=y)

# Define the range of depth limits and number of runs to average over
depths = [1, 2, 3, 4, 5, 6, 8, 10]
runs = 10

# Arrays to keep Acc Values
train_acc = np.zeros((runs, len(depths))) # This is going to be a 10x8 array, because we want to keep all the accuracies for each run
test_acc = np.zeros((runs, len(depths)))

# Loop over the runs
for j, depth in enumerate(depths): # Need enumerate since we need to index the depths to keep in arrays
    for i in range(runs):
        # Learn Classifier
        predictor = tree.DecisionTreeClassifier(max_depth=depth)
        predictor.fit(X_train, y_train)

        # Predict on train and test set
        y_predicted_train = predictor.predict(X_train)
        y_predicted_test = predictor.predict(X_test)

        # Compute accuracy
        train_acc[i, j] = sk.metrics.accuracy_score(y_train, y_predicted_train)
        test_acc[i, j] = sk.metrics.accuracy_score(y_test, y_predicted_test)

# Compute mean and standard deviation for train and test accuracies
train_acc_mean = np.mean(train_acc, axis=0)
train_acc_std = np.std(train_acc, axis=0)
test_acc_mean = np.mean(test_acc, axis=0)
test_acc_std = np.std(test_acc, axis=0)

# Plot Optional
plt.figure(figsize=(10, 5))
plt.title('Accuracy vs. Depth Limit with 10 runs per depth limit')
plt.errorbar(depths, train_acc_mean, yerr=train_acc_std, label='Train Accuracy')
plt.errorbar(depths, test_acc_mean, yerr=test_acc_std, label='Test Accuracy')
plt.xlabel('Depth Limit')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(IMAGES_DIR /'accuracy_vs_depth_limit.png')
plt.show()


# Plot the Exercise without Optional Part
plt.figure(figsize=(10, 5))
plt.title('Accuracy vs. Depth Limit with 1 runs per depth limit')
plt.plot(depths, train_acc[0], label='Train Accuracy') # Selecting the first run for each depth limit
plt.plot(depths, test_acc[0], label='Test Accuracy')
plt.xlabel('Depth Limit')
plt.ylabel('Accuracy')
plt.legend()
plt.savefig(IMAGES_DIR /'accuracy_vs_depth_limit_without_optional.png')
plt.show()


: 

# Question 3

<!-- As seen by both plots, the training accuracy increases as the depth limit of the decision tree increases. However, the testing accuracy increases until a certain depth limit (in this case around 4 or 5) and then starts to decrease. This is because the decision tree is overfitting the training data and is not generalizing well to the testing data.
For the optional part, we can also see that the standard deviation is generally small for the training, indicating that the results are consistent across different random initializations of the data. However, the standard deviation is larger for the testing data, indicating that the testing data is more sensitive to the random initialization of the data. -->

Como se vê em ambos os gráficos, a precisão de treino aumenta à medida que o limite de profundidade da árvore de decisão aumenta. No entanto, a precisão de teste aumenta até um certo limite de profundidade (neste caso, cerca de 4 ou 5) e depois começa a diminuir. Isto acontece porque a árvore de decisão está a dar overfit aos dados de treino e não está a aprender de modo a corretamente avaliar os dados de teste.
No que toca à parte opcional, também podemos ver que o desvio padrão geralmente é pequeno no caso do treino, o que indica que os resultados são consistentes em diferentes inicializações aleatórias dos dados. No entanto, o desvio padrão é maior para os dados de teste, o que indica que os dados de teste são mais sensíveis à inicialização aleatória dos dados.

# Question 4

In [ ]:
# Learn a decision tree with a minimum number of samples per leaf = 20
predictor = tree.DecisionTreeClassifier(min_samples_leaf=20,
                                        random_state=0)

# Fit all the data
predictor.fit(X, y)

# Plot the tree
plt.figure(figsize=(20, 10))
tree.plot_tree(predictor,
                feature_names=X.columns,
                filled=True,
                class_names=predictor.classes_)
plt.title('Decision Tree with Minimum Samples per Leaf = 20', fontsize=20)
plt.savefig(IMAGES_DIR / 'decision_tree_min_samples_leaf_20.png')
plt.show()

: 

### Question 4 ii)


De acordo com a árvore de decisão construída na alínea anterior, existem dois casos nos quais o indivíduo em questão tem um outcome de diagnóstico com Hérnia. Em primeiro lugar temos degree_spondy|olisthesis $\leq 16.079$ e sacral_slope $\leq 28.136$. 
Em segundo lugar, temos degree_spondy|olisthesis $\leq 16.079$, sacral_slope $> 28.136$, pelvic_radius $ \leq 117.36$ e sacral_slope $\leq 40.149$.